# "Here I Created The Date Dimension for Time intelligence purposes"

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("SampleDataToHive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.jars", "/Drivers/SQL_Sever/jdbc/sqljdbc42.jar")\
    .enableHiveSupport() \
    .getOrCreate()


In [3]:
orderDate =spark.read.format("jdbc")\
        .option("url", "jdbc:sqlserver://172.18.0.4:1433;databaseName=AdventureWorks2017")\
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("dbtable", "[Sales].[SalesOrderHeader]")\
        .option("user","sa")\
        .option("password", "Mo*012105")\
        .load()


In [4]:
order_dates = orderDate.select("OrderDate").distinct()

#Here i build the Date Dimension based on Order Date.

In [11]:
DimDate = order_dates.select(
    col("OrderDate").alias("DateKey"),
    date_format("OrderDate", "yyyy-MM-dd").alias("Date"),
    date_format("OrderDate", "yyyy").alias("Year"),
    date_format("OrderDate", "MM").alias("Month"),
    date_format("OrderDate", "dd").alias("Day"),
    quarter("OrderDate").alias("Quarter"))\
    .repartition(4, "DateKey")

# Show the dimDate DataFrame
DimDate.show()
DimDate.write.format("hive").mode("overwrite").saveAsTable("bronze.DimDate")

+-------------------+----------+----+-----+---+-------+
|            DateKey|      Date|Year|Month|Day|Quarter|
+-------------------+----------+----+-----+---+-------+
|2011-10-10 00:00:00|2011-10-10|2011|   10| 10|      4|
|2011-12-04 00:00:00|2011-12-04|2011|   12| 04|      4|
|2012-01-22 00:00:00|2012-01-22|2012|   01| 22|      1|
|2012-07-11 00:00:00|2012-07-11|2012|   07| 11|      3|
|2012-10-20 00:00:00|2012-10-20|2012|   10| 20|      4|
|2013-10-31 00:00:00|2013-10-31|2013|   10| 31|      4|
|2011-07-21 00:00:00|2011-07-21|2011|   07| 21|      3|
|2011-08-26 00:00:00|2011-08-26|2011|   08| 26|      3|
|2012-05-12 00:00:00|2012-05-12|2012|   05| 12|      2|
|2013-02-12 00:00:00|2013-02-12|2013|   02| 12|      1|
|2013-10-11 00:00:00|2013-10-11|2013|   10| 11|      4|
|2013-11-21 00:00:00|2013-11-21|2013|   11| 21|      4|
|2012-10-10 00:00:00|2012-10-10|2012|   10| 10|      4|
|2012-12-18 00:00:00|2012-12-18|2012|   12| 18|      4|
|2013-05-27 00:00:00|2013-05-27|2013|   05| 27| 

In [6]:
spark.sql("show databases").show()

+-----------------+
|        namespace|
+-----------------+
|bronzesalesschema|
|          default|
|      my_database|
|            sales|
|             test|
+-----------------+



In [7]:
spark.sql("use bronzesalesschema")

DataFrame[]

In [8]:
spark.sql("show tables").show()

+-----------------+-----------+-----------+
|        namespace|  tableName|isTemporary|
+-----------------+-----------+-----------+
|bronzesalesschema|dimcustomer|      false|
|bronzesalesschema|    dimdate|      false|
+-----------------+-----------+-----------+

